In [1]:
#установка библиотек
!pip install -U psycopg2
!pip install -U plotly

In [2]:
import psycopg2
psycopg2.__version__

'2.9.3 (dt dec pq3 ext lo64)'

# проверка подключения

In [3]:
# student11
# student11_password

In [4]:
#проверка подключения PostgreSQL 
import psycopg2
import pandas as pd
#Библиотека ждя визуализации
from IPython.display import HTML
import plotly.express as px


#!введите свои реквизиты!
DB_HOST = '52.157.159.24'
DB_USER = 'student11'
DB_USER_PASSWORD = 'student11_password'
DB_NAME = 'sql_ex_third_stream'

conn = psycopg2.connect(host=DB_HOST, user=DB_USER, password=DB_USER_PASSWORD, dbname=DB_NAME)

In [5]:
conn

<connection object at 0x7f1eb350ac20; dsn: 'user=student11 password=xxx dbname=sql_ex_third_stream host=52.157.159.24', closed: 0>

In [6]:
conn.isexecuting()

False

# задания












# task13 (lesson3)
* Компьютерная фирма: Вывести список всех продуктов и производителя с указанием типа продукта (pc, printer, laptop). Вывести: model, maker, type

In [7]:
SQL1 =  """
            SELECT model, maker, type
            FROM product
        """

df = pd.read_sql_query(SQL1, conn)
df


,model,maker,type
0,1232,A,PC
1,1233,A,PC
2,1260,E,PC
3,1276,A,Printer
4,1288,D,Printer
5,1298,A,Laptop
6,1321,C,Laptop
7,1401,A,Printer
8,1408,A,Printer
9,1433,D,Printer


# task14 (lesson3)
* Компьютерная фирма: При выводе всех значений из таблицы printer дополнительно вывести для тех, у кого цена вышей средней PC - "1", у остальных - "0"

In [8]:
SQL1 =  """
            SELECT *,
            CASE 
                WHEN price > (SELECT AVG(price) FROM PC) 
                    THEN 1
                    ELSE 0
            END flag
            FROM printer
        """

df = pd.read_sql_query(SQL1, conn)
df


,code,model,color,type,price,flag
0,1,1276,n,Laser,400.0,0
1,2,1433,y,Jet,270.0,0
2,3,1434,y,Jet,290.0,0
3,4,1401,n,Matrix,150.0,0
4,5,1408,n,Matrix,270.0,0
5,6,1288,n,Laser,400.0,0


# task15 (lesson3)
* Корабли: Вывести список кораблей, у которых class отсутствует (IS NULL)

In [9]:
SQL1 =  """
            SELECT *
            FROM ships
            WHERE class IS NULL
        """

df = pd.read_sql_query(SQL1, conn)
df


,name,class,launched


# task16 (lesson3)
* Корабли: Укажите сражения, которые произошли в годы, не совпадающие ни с одним из годов спуска кораблей на воду.

In [10]:
SQL1 =  """
            WITH battle_with_year AS
            (
                SELECT *, EXTRACT(year FROM date) AS battle_year
                FROM battles
            )
            SELECT * FROM battle_with_year
            WHERE battle_year NOT IN (SELECT launched FROM ships)
        """

df = pd.read_sql_query(SQL1, conn)
df


,name,date,battle_year
0,#Cuba62a,1962-10-20,1962.0
1,#Cuba62b,1962-10-25,1962.0


# task17 (lesson3)
* Корабли: Найдите сражения, в которых участвовали корабли класса Kongo из таблицы Ships.

In [11]:
SQL1 =  """
            SELECT battle 
            FROM outcomes
            INNER JOIN ships
            ON outcomes.ship = ships.name
            WHERE ships.class = 'Kongo'
        """

df = pd.read_sql_query(SQL1, conn)
df


,battle
0,Guadalcanal


# task1  (lesson4)
* Компьютерная фирма: Сделать view (название all_products_flag_300) для всех товаров (pc, printer, laptop) с флагом, если стоимость больше > 300. Во view три колонки: model, price, flag

In [28]:
SQL1 =  """
            DROP VIEW IF EXISTS all_products_flag_300;

            CREATE VIEW all_products_flag_300 AS 
            (
                SELECT model, price, 
                CASE
                    WHEN price > 300 
                    THEN 1
                    ELSE 0
                END flag
                FROM pc

                UNION 
                SELECT model, price, 
                CASE
                    WHEN price > 300 
                    THEN 1
                    ELSE 0
                END flag
                FROM printer

                UNION 
                SELECT model, price, 
                CASE
                    WHEN price > 300 
                    THEN 1
                    ELSE 0
                END flag
                FROM laptop
            )
            ;

            SELECT * 
            FROM all_products_flag_300;
        """

df = pd.read_sql_query(SQL1, conn)
df


,model,price,flag
0,1233,600.0,1
1,1401,150.0,0
2,1298,700.0,1
3,1408,270.0,0
4,1321,970.0,1
5,1288,400.0,1
6,1298,1050.0,1
7,1233,970.0,1
8,1434,290.0,0
9,1752,1150.0,1


# task2  (lesson4)
* Компьютерная фирма: Сделать view (название all_products_flag_avg_price) для всех товаров (pc, printer, laptop) с флагом, если стоимость больше cредней . Во view три колонки: model, price, flag

In [29]:
SQL1 =  """
            DROP VIEW IF EXISTS all_products_flag_avg_price;

            CREATE VIEW all_products_flag_avg_price AS 
            (
                SELECT model, price, 
                CASE
                    WHEN price > (SELECT AVG(price) FROM pc) 
                    THEN 1
                    ELSE 0
                END flag
                FROM pc

                UNION 
                SELECT model, price, 
                CASE
                    WHEN price > (SELECT AVG(price) FROM printer) 
                    THEN 1
                    ELSE 0
                END flag
                FROM printer

                UNION 
                SELECT model, price, 
                CASE
                    WHEN price > (SELECT AVG(price) FROM laptop) 
                    THEN 1
                    ELSE 0
                END flag
                FROM laptop
            )
            ;

            SELECT * 
            FROM all_products_flag_avg_price;
        """

df = pd.read_sql_query(SQL1, conn)
df


,model,price,flag
0,1401,150.0,0
1,1232,350.0,0
2,1408,270.0,0
3,1288,400.0,1
4,1298,1050.0,1
5,1233,970.0,1
6,1434,290.0,0
7,1232,400.0,0
8,1752,1150.0,1
9,1233,980.0,1


# task3  (lesson4)
* Компьютерная фирма: Вывести все принтеры производителя = 'A' со стоимостью выше средней по принтерам производителя = 'D' и 'C'. Вывести model

In [14]:
SQL1 =  """
            WITH avg_price AS
            (
                SELECT AVG(price) AS avg_price
                FROM product
                INNER JOIN printer
                ON product.model = printer.model
                WHERE maker = 'C' OR maker = 'D'
            )
            SELECT product.model
            FROM product
            INNER JOIN printer
            ON product.model = printer.model
            WHERE maker = 'A' AND price > (SELECT avg_price FROM avg_price)
        """

df = pd.read_sql_query(SQL1, conn)
df


,model
0,1276


# task4 (lesson4)
* Компьютерная фирма: Вывести все товары производителя = 'A' со стоимостью выше средней по принтерам производителя = 'D' и 'C'. Вывести model

In [15]:
SQL1 =  """
            WITH avg_price AS
            (
                SELECT AVG(price) AS avg_price
                FROM product
                INNER JOIN printer
                ON product.model = printer.model
                WHERE maker = 'C' OR maker = 'D'
            ),
            all_maker_A AS
            (
                SELECT pc.model, pc.price 
                FROM product
                INNER JOIN pc
                ON product.model = pc.model
                WHERE maker = 'A' 

                UNION 
                SELECT printer.model, printer.price
                FROM product
                INNER JOIN printer
                ON product.model = printer.model
                WHERE maker = 'A' 

                UNION 
                SELECT laptop.model, laptop.price
                FROM product
                INNER JOIN laptop
                ON product.model = laptop.model
                WHERE maker = 'A' 
            )
            SELECT DISTINCT model
            FROM all_maker_A
            WHERE price > (SELECT avg_price FROM avg_price)
        """

df = pd.read_sql_query(SQL1, conn)
df


,model
0,1233
1,1232
2,1752
3,1298
4,1276


# task5 (lesson4)
* Компьютерная фирма: Какая средняя цена среди уникальных продуктов производителя = 'A' (printer & laptop & pc)

In [16]:
SQL1 =  """
            WITH all_maker_A AS
            (
                SELECT pc.model, pc.price 
                FROM product
                INNER JOIN pc
                ON product.model = pc.model
                WHERE maker = 'A' 

                UNION 
                SELECT printer.model, printer.price
                FROM product
                INNER JOIN printer
                ON product.model = printer.model
                WHERE maker = 'A' 

                UNION 
                SELECT laptop.model, laptop.price
                FROM product
                INNER JOIN laptop
                ON product.model = laptop.model
                WHERE maker = 'A' 
            )
            SELECT AVG(price)
            FROM all_maker_A
        """

df = pd.read_sql_query(SQL1, conn)
df


,avg
0,680.0


# task6 (lesson4)
* Компьютерная фирма: Сделать view с количеством товаров (название count_products_by_makers) по каждому производителю. Во view: maker, count

In [17]:
SQL1 =  """
            DROP VIEW IF EXISTS count_products_by_makers;

            CREATE VIEW count_products_by_makers AS
                SELECT maker, COUNT(1)
                FROM product
                GROUP BY maker;

            SELECT * 
            FROM count_products_by_makers
        """

df = pd.read_sql_query(SQL1, conn)
df


,maker,count
0,E,7
1,D,2
2,A,7
3,B,2
4,C,1


# task7 (lesson4)
* По предыдущему view (count_products_by_makers) сделать график в colab (X: maker, y: count)

In [18]:
SQL1 =  """
            DROP VIEW IF EXISTS count_products_by_makers;

            CREATE VIEW count_products_by_makers AS
                SELECT maker, COUNT(1)
                FROM product
                GROUP BY maker;

            SELECT * 
            FROM count_products_by_makers
        """

df = pd.read_sql_query(SQL1, conn)
df


fig = px.bar(x=df['maker'].to_list(), y=df['count'].to_list(), labels={'x':'maker', 'y':'count'})
fig.show()

# task8 (lesson4)
* Компьютерная фирма: Сделать копию таблицы printer (название printer_updated) и удалить из нее все принтеры производителя 'D'

In [30]:
SQL1 =  """
            DROP TABLE IF EXISTS printer_updated_11;

            CREATE TABLE printer_updated_11 AS
                SELECT * 
                FROM printer
                WHERE model not in (SELECT model FROM product WHERE maker = 'D')
            ;

            SELECT * FROM printer_updated_11;
        """

df = pd.read_sql_query(SQL1, conn)
df


,code,model,color,type,price
0,1,1276,n,Laser,400.0
1,3,1434,y,Jet,290.0
2,4,1401,n,Matrix,150.0
3,5,1408,n,Matrix,270.0


# task9 (lesson4)
* Компьютерная фирма: Сделать на базе таблицы (printer_updated) view с дополнительной колонкой производителя (название printer_updated_with_makers)

In [31]:
SQL1 =  """
            DROP TABLE IF EXISTS printer_updated_11;

            CREATE TABLE printer_updated_11 AS
                SELECT * 
                FROM printer
                WHERE model not in (SELECT model FROM product WHERE maker = 'D')
            ;
            
            DROP VIEW IF EXISTS printer_updated_with_makers_11;

            CREATE VIEW printer_updated_with_makers_11 AS
                SELECT printer_updated_11.*, product.maker
                FROM printer_updated_11
                INNER JOIN product
                ON printer_updated_11.model = product.model
            ;

            SELECT * FROM printer_updated_with_makers_11;
        """

df = pd.read_sql_query(SQL1, conn)
df


,code,model,color,type,price,maker
0,1,1276,n,Laser,400.0,A
1,3,1434,y,Jet,290.0,E
2,4,1401,n,Matrix,150.0,A
3,5,1408,n,Matrix,270.0,A


# task10 (lesson4)
* Корабли: Сделать view c количеством потопленных кораблей и классом корабля (название sunk_ships_by_classes). Во view: count, class (если значения класса нет/IS NULL, то заменить на 0)

In [21]:
SQL1 =  """
            DROP VIEW IF EXISTS sunk_ships_by_classes;
            DROP VIEW IF EXISTS sunk_ships_by_classes_corrected;

            CREATE VIEW sunk_ships_by_classes_corrected AS
                SELECT COUNT(1) AS count, 
                CASE
                    WHEN class IS NOT NULL
                        THEN class
                        ELSE '0'
                    end class_corrected
                FROM outcomes
                LEFT JOIN ships
                ON outcomes.ship = ships.name
                WHERE result = 'sunk'
                GROUP BY class_corrected
            ;

            CREATE VIEW sunk_ships_by_classes AS
                SELECT count, class_corrected AS class
                FROM sunk_ships_by_classes_corrected
            ;

            SELECT * FROM sunk_ships_by_classes;
        """

df = pd.read_sql_query(SQL1, conn)
df


,count,class
0,5,0
1,1,Kongo


# task11 (lesson4)
* Корабли: По предыдущему view (sunk_ships_by_classes) сделать график в colab (X: class, Y: count)

In [22]:
SQL1 =  """
            DROP VIEW IF EXISTS sunk_ships_by_classes;
            DROP VIEW IF EXISTS sunk_ships_by_classes_corrected;

            CREATE VIEW sunk_ships_by_classes_corrected AS
                SELECT COUNT(1) AS count, 
                CASE
                    WHEN class IS NOT NULL
                        THEN class
                        ELSE '0'
                    end class_corrected
                FROM outcomes
                LEFT JOIN ships
                ON outcomes.ship = ships.name
                WHERE result = 'sunk'
                GROUP BY class_corrected
            ;

            CREATE VIEW sunk_ships_by_classes AS
                SELECT count, class_corrected AS class
                FROM sunk_ships_by_classes_corrected
            ;

            SELECT * FROM sunk_ships_by_classes;
        """

df = pd.read_sql_query(SQL1, conn)
df


fig = px.bar(x=df['class'].to_list(), y=df['count'].to_list(), labels={'x':'class', 'y':'count'})
fig.show()

# task12 (lesson4)
* Корабли: Сделать копию таблицы classes (название classes_with_flag) и добавить в нее flag: если количество орудий больше или равно 9 - то 1, иначе 0

In [23]:
SQL1 =  """
            DROP TABLE IF EXISTS classes_with_flag;

            CREATE TABLE classes_with_flag AS
                SELECT *, 
                CASE
                    WHEN numguns >= 9
                        THEN 1
                        ELSE 0
                    end flag
                FROM classes
            ;

            SELECT * FROM classes_with_flag;
        """

df = pd.read_sql_query(SQL1, conn)
df



,class,type,country,numguns,bore,displacement,flag
0,Bismarck,bb,Germany,8,15.0,42000,0
1,Iowa,bb,USA,9,16.0,46000,1
2,Kongo,bc,Japan,8,14.0,32000,0
3,North Carolina,bb,USA,12,16.0,37000,1
4,Renown,bc,Gt.Britain,6,15.0,32000,0
5,Revenge,bb,Gt.Britain,8,15.0,29000,0
6,Tennessee,bb,USA,12,14.0,32000,1
7,Yamato,bb,Japan,9,18.0,65000,1


# task13 (lesson4)
* Корабли: Сделать график в colab по таблице classes с количеством классов по странам (X: country, Y: count)

In [24]:
SQL1 =  """
            SELECT country, COUNT(1) AS count
            FROM classes
            GROUP BY country;
        """

df = pd.read_sql_query(SQL1, conn)
df


fig = px.bar(x=df['country'].to_list(), y=df['count'].to_list(), labels={'x':'country', 'y':'count'})
fig.show()

# task14 (lesson4)
* Корабли: Вернуть количество кораблей, у которых название начинается с буквы "O" или "M".

In [25]:
SQL1 =  """
            SELECT COUNT(1) AS count
            FROM ships
            WHERE (name LIKE 'O%') OR (name LIKE 'M%');
        """

df = pd.read_sql_query(SQL1, conn)
df


,count
0,2


# task15 (lesson4)
* Корабли: Вернуть количество кораблей, у которых название состоит из двух слов.

In [26]:
SQL1 =  """
            SELECT COUNT(1) AS count
            FROM ships
            WHERE (name LIKE '% %');
        """

df = pd.read_sql_query(SQL1, conn)
df



,count
0,5


# task16 (lesson4)
* Корабли: Построить график с количеством запущенных на воду кораблей и годом запуска (X: year, Y: count)

In [27]:
SQL1 =  """
            SELECT launched AS year, COUNT(1) AS count
            FROM ships
            GROUP BY year
            ;
        """

df = pd.read_sql_query(SQL1, conn)
df

fig = px.bar(x=df['year'].to_list(), y=df['count'].to_list(), labels={'x':'year', 'y':'count'})
fig.show()